In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

# 设置中文字体正常
sns.set(font="SimHei")
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

C:\ProgramData\Miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\ProgramData\Miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# 读数据
result1 = pd.read_excel('correlation_results.xlsx')
result2 = pd.read_excel('correlation_results2.xlsx')
result1

,Indicator 1,Indicator 2,Spearman Correlation,P-Value
0,性别,年龄,0.022558,7.297308e-01
1,性别,糖尿病,0.087035,1.817675e-01
2,性别,高血压,0.115637,7.560672e-02
3,性别,乙肝,0.052805,4.184026e-01
4,性别,M,0.081363,2.120235e-01
...,...,...,...,...
625,AST,TC,-0.041759,5.223322e-01
626,AST,eGFR,-0.110447,8.978705e-02
627,TG,TC,0.241113,1.784620e-04
628,TG,eGFR,-0.356321,1.673236e-08


In [44]:
# 画网络图
def plt_network(data, s_max, color_li, file):
    # 生成指标列表
    li11 = []
    for i in data['Indicator 1'].tolist():
        if i not in li11:
            li11.append(i)
    li12 = []
    for i in data['Indicator 2'].tolist():
        if i not in li12:
            li12.append(i)
    li11.append(li12[-1])
    
    # 初始化网络二维列表
    num = [[0 for i in range(1,len(li11)+1)]for j in range(1,len(li11)+1)]
    
    # 根据指标值修改网络图中的指标数值
    for i in li11[:-1]:
        for j in li11[1:]:
            if not data[(data['Indicator 1']==i) & (data['Indicator 2']==j)].empty:
                if (float(data[(data['Indicator 1']==i) & (data['Indicator 2']==j)]['Spearman Correlation']) > s_max) or (float(data[(data['Indicator 1']==i) & (data['Indicator 2']==j)]['Spearman Correlation']) < s_max*(-1)):
                    num[li11.index(i)][li11.index(j)] = float(data[(data['Indicator 1']==i) & (data['Indicator 2']==j)]['Spearman Correlation'])
    # 定义网络矩阵
    Matrix = np.array(num)
    G = nx.Graph()
    li_temp = []
    elarge = []
    esmall = []
    
    # 初始化graph_edges
    for i in range(len(Matrix)):
        for j in range(len(Matrix)):
            if Matrix[i, j] > 0:
                G.add_edge(li11[i], li11[j], weight=Matrix[i, j]*10000)
                elarge.append((li11[i], li11[j]))
                li_temp.append(li11[i])
                li_temp.append(li11[j])
            if Matrix[i, j] < 0:
                G.add_edge(li11[i], li11[j], weight=-Matrix[i, j]*10000)
                esmall.append((li11[i], li11[j]))
                li_temp.append(li11[i])
                li_temp.append(li11[j])
    
    # 添加节点
    li_ = list(set(li_temp))
#     print(li_)
    G.add_nodes_from(li_)
    
    # 定义布局
#     pos = nx.kamada_kawai_layout(G)
    pos = nx.random_layout(G)
    
    # 画出网络图
    nx.draw_networkx_nodes(G, pos, node_size=200, node_color=color_li[0])
    nx.draw_networkx_edges(G, pos, edgelist=esmall, width=1, edge_color=color_li[1], alpha=0.5, style="dashed")
    nx.draw_networkx_edges(G, pos, edgelist=elarge, width=1, edge_color=color_li[2])
    nx.draw_networkx_labels(G, pos, font_size=4, font_color=color_li[3])
    
    # 调整图形样式
    ax = plt.gca()
    ax.margins(0.08)
    plt.axis("off")
    plt.tight_layout()
    
    # 保存文件
    plt.savefig(file, dpi=1000, bbox_inches='tight') 
    plt.show()
    
    # 返回edges_num与mean_degree
    edges_num = len(G.edges)
    mean_degree = sum(dict(G.degree()).values())/len(G)
    return edges_num, mean_degree, G